## This notebook is for finding highly misclassified data points.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py

from astropy.io import fits
from glob import glob

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer, confusion_matrix
from sklearn import warnings

from IPython.display import SVG
from keras.utils.vis_utils import plot_model

from keras import backend as K
from keras.models import Sequential
from keras.layers import Input,Dense, Activation, Flatten, Convolution1D, Dropout, MaxPooling1D, Conv2D, Conv1D, InputLayer, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard
from keras.constraints import Constraint
from keras.initializers import Initializer, glorot_uniform
from keras.utils import np_utils
from keras.layers import Concatenate
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Model

import tensorflow as tf

from Modeling import CrossValidationModeler

Using TensorFlow backend.


Modles to use for evaluation

In [33]:
def create_conv_flux_model_eboss():
    model = Sequential()
    model.add(InputLayer(batch_input_shape=(None,4563, 1)))
    model.add(Conv1D(filters=64, kernel_size=2,activation= 'relu'))
    model.add(Conv1D(filters=24, kernel_size=2,activation= 'relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(30,activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(20, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(.00001), metrics=['accuracy'])
    return model

# def create_conv_flux_model():
#     model = Sequential()
#     model.add(InputLayer(batch_input_shape=(None,4563, 1)))
#     model.add(Conv1D(filters=64, kernel_size=2,activation= 'relu'))
#     model.add(Conv1D(filters=24, kernel_size=2,activation= 'relu'))
#     model.add(MaxPooling1D())
#     model.add(Flatten())
#     model.add(Dense(30,activation='relu'))
#     model.add(Dropout(.5))
#     model.add(Dense(20, activation='relu'))
#     model.add(Dropout(.5))
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer=Adam(lr=.00001), metrics=['accuracy'])
#     return model

def create_conv_flux_model_manga():
    model = Sequential()
    model.add(InputLayer(batch_input_shape=(None,4563, 1)))
    model.add(Conv1D(filters=64, kernel_size=2,activation= 'relu'))
    model.add(Conv1D(filters=24, kernel_size=2,activation= 'relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(30,activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(20, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=.001), metrics=['accuracy'])
    return model

eboss, or manga data

In [31]:
eboss = h5py.File('../../../flux_data/manga_flux_multiline+.hdf5')
X, Y = np.array(eboss['flux_values']), np.array(eboss['flux_labels'])
res_flux = np.array(eboss['res_flux_values'])
ivar = np.array(eboss['ivar_flux_values'])
ids = np.array(eboss['flux_ids'])

res_flux = np.expand_dims(res_flux, -1)
ivar = np.expand_dims(ivar, -1)
X = np.expand_dims(X, -1)
eboss.close()

Initial testing to ensure model architecture is correct.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(res_flux, Y)
create_conv_flux_model_manga().fit(X_train, y_train, epochs=25, batch_size=256, validation_data=(X_test, y_test), verbose=1)

Train on 1638 samples, validate on 546 samples
Epoch 1/25
1638/1638 [==============================] - 1s 691us/step - loss: 0.8376 - acc: 0.5317 - val_loss: 0.6483 - val_acc: 0.7766
Epoch 2/25
1638/1638 [==============================] - 0s 160us/step - loss: 0.6938 - acc: 0.6496 - val_loss: 0.6705 - val_acc: 0.8077
Epoch 3/25
1638/1638 [==============================] - 0s 159us/step - loss: 0.6671 - acc: 0.6880 - val_loss: 0.6267 - val_acc: 0.8828
Epoch 4/25
1638/1638 [==============================] - 0s 159us/step - loss: 0.6207 - acc: 0.6862 - val_loss: 0.5265 - val_acc: 0.8516
Epoch 5/25
1638/1638 [==============================] - 0s 158us/step - loss: 0.5653 - acc: 0.7332 - val_loss: 0.4989 - val_acc: 0.8535
Epoch 6/25
1638/1638 [==============================] - 0s 158us/step - loss: 0.5467 - acc: 0.7179 - val_loss: 0.3971 - val_acc: 0.8736
Epoch 7/25
1638/1638 [==============================] - 0s 157us/step - loss: 0.5148 - acc: 0.7314 - val_loss: 0.3757 - val_acc: 0.8828
E

# Create a bunch of models

In [34]:
mods = [create_conv_flux_model_manga() for _ in range(75)]
for ix, mod in enumerate(mods):
    print('Model {}'.format(ix))
    X_train, X_test, y_train, y_test = train_test_split(res_flux, Y)
    
    # Uncomment if eboss
    #mod.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test), verbose=0)
    
    # Comment if manga.
    mod.fit(X_train, y_train, epochs=25, batch_size=256, validation_data=(X_test, y_test), verbose=0)

Model 0
Model 1
Model 2
Model 3
Model 4
Model 5
Model 6
Model 7
Model 8
Model 9
Model 10
Model 11
Model 12
Model 13
Model 14
Model 15
Model 16
Model 17
Model 18
Model 19
Model 20
Model 21
Model 22
Model 23
Model 24
Model 25
Model 26
Model 27
Model 28
Model 29
Model 30
Model 31
Model 32
Model 33
Model 34
Model 35
Model 36
Model 37
Model 38
Model 39
Model 40
Model 41
Model 42
Model 43
Model 44
Model 45
Model 46
Model 47
Model 48
Model 49
Model 50
Model 51
Model 52
Model 53
Model 54
Model 55
Model 56
Model 57
Model 58
Model 59
Model 60
Model 61
Model 62
Model 63
Model 64
Model 65
Model 66
Model 67
Model 68
Model 69
Model 70
Model 71
Model 72
Model 73
Model 74


## Extract the misclassified labels

In [35]:
dict_stor = {}
bad_sets = [set() for _ in range(len(mods))]
for ix, (mod, bad_set) in enumerate(zip(mods, bad_sets)):
    print('Model Number: {}'.format(ix+1))
    preds_classes = mod.predict_classes(res_flux)
    preds_probas = mod.predict(res_flux)
    for pred_class, pred_proba, label, id in zip(preds_classes, preds_probas, Y, ids):
        if pred_class != label:
            dict_stor[(id[0], ix+1)] = (id[0], label[0], pred_class[0], pred_proba[0])
            bad_set.add(id[0])
            #print("Predicted {} when it was really {}. ID: {}, PROBA: {}".format(pred_class, label, id, pred_proba))

    print(confusion_matrix(Y, mod.predict_classes(res_flux)))
    print('-----------------------------------------------')

Model Number: 1
[[ 720   60]
 [  24 1380]]
-----------------------------------------------
Model Number: 2
[[ 726   54]
 [  22 1382]]
-----------------------------------------------
Model Number: 3
[[ 695   85]
 [  26 1378]]
-----------------------------------------------
Model Number: 4
[[ 722   58]
 [  19 1385]]
-----------------------------------------------
Model Number: 5
[[ 708   72]
 [  15 1389]]
-----------------------------------------------
Model Number: 6
[[ 710   70]
 [  34 1370]]
-----------------------------------------------
Model Number: 7
[[ 695   85]
 [  16 1388]]
-----------------------------------------------
Model Number: 8
[[ 725   55]
 [  15 1389]]
-----------------------------------------------
Model Number: 9
[[ 715   65]
 [  17 1387]]
-----------------------------------------------
Model Number: 10
[[ 746   34]
 [  28 1376]]
-----------------------------------------------
Model Number: 11
[[ 742   38]
 [  14 1390]]
---------------------------------------------

Make a dataframe from the inspection points gathered

In [3]:
# # Use with manga
# inspection_points = pd.DataFrame.from_dict(dict_stor, orient='index', columns=['id', 'label', 'pred_label', 'proba'])
# inspection_points.to_csv('manga_inspection.csv')
# inspection_points

inspection_points = pd.read_csv('manga_inspection.csv')
inspection_points

,Unnamed: 0,id,label,pred_label,proba
0,"(b'8149_12702_fiber_669_multiline', 1)",b'8149_12702_fiber_669_multiline',0.0,1,0.991832
1,"(b'8155_6104_fiber_157_multiline', 1)",b'8155_6104_fiber_157_multiline',0.0,1,0.595219
2,"(b'8155_6104_fiber_460_multiline', 1)",b'8155_6104_fiber_460_multiline',0.0,1,0.947801
3,"(b'8253_9101_fiber_1140_multiline', 1)",b'8253_9101_fiber_1140_multiline',0.0,1,0.728340
4,"(b'8254_12704_fiber_914_multiline', 1)",b'8254_12704_fiber_914_multiline',0.0,1,0.989431
5,"(b'8255_9102_fiber_325_multiline', 1)",b'8255_9102_fiber_325_multiline',0.0,1,0.626767
6,"(b'8258_6101_fiber_588_multiline', 1)",b'8258_6101_fiber_588_multiline',0.0,1,0.613627
7,"(b'8261_12705_fiber_1038_multiline', 1)",b'8261_12705_fiber_1038_multiline',0.0,1,0.955092
8,"(b'8263_12704_fiber_212_multiline', 1)",b'8263_12704_fiber_212_multiline',0.0,1,0.989697
9,"(b'8315_3701_fiber_197_multiline', 1)",b'8315_3701_fiber_197_multiline',0.0,1,0.989994


In [38]:
#inspection_points = pd.DataFrame.from_dict(dict_stor, orient='index', columns=['id', 'label', 'pred_label', 'proba'])
#inspection_points.to_csv('eboss_inspection.csv')
# inspection_points

# inspection_points = pd.read_csv('eboss_inspection.csv')
# inspection_points

In [4]:
len(inspection_points)

5974

# Set up a dataframe for easy analysis

In [6]:
should_be_0 = inspection_points.label == 0
should_be_1 = inspection_points.label == 1
greater_than_90 = inspection_points.proba > .9
less_than_10 = inspection_points.proba < .1

# False Positives (Pred > .9) 

In [9]:
# Printing out the first 20 results in a list easily copied into word.
for t in list(false_positives.iteritems())[:20]:
    splitted = str(t[0]).split('_')
    splitted = splitted[:2] + splitted[3:4]
    print(' '.join(splitted).lstrip("'b") + ' ' + str(t[1]))

8132 9102 958 29
8606 6101 173 27
8263 12704 212 27
8254 12704 914 23
8947 6102 475 21
8465 12701 120 20
8131 3704 334 20
8131 6102 419 20
8726 9101 758 19
8083 9101 1334 19
8948 12701 175 19
7990 6101 321 19
8728 6104 683 18
8084 12701 1097 18
8149 12702 669 18
8604 9101 614 18
8482 12701 873 18
8155 6104 157 18
8566 9102 958 18
8338 3702 220 18


In [7]:
false_positives = inspection_points[greater_than_90 & should_be_0].id.value_counts()
false_positives

b'8132_9102_fiber_958_multiline'      29
b'8606_6101_fiber_173_multiline'      27
b'8263_12704_fiber_212_multiline'     27
b'8254_12704_fiber_914_multiline'     23
b'8947_6102_fiber_475_multiline'      21
b'8465_12701_fiber_120_multiline'     20
b'8131_3704_fiber_334_multiline'      20
b'8131_6102_fiber_419_multiline'      20
b'8726_9101_fiber_758_multiline'      19
b'8083_9101_fiber_1334_multiline'     19
b'8948_12701_fiber_175_multiline'     19
b'7990_6101_fiber_321_multiline'      19
b'8728_6104_fiber_683_multiline'      18
b'8084_12701_fiber_1097_multiline'    18
b'8149_12702_fiber_669_multiline'     18
b'8604_9101_fiber_614_multiline'      18
b'8482_12701_fiber_873_multiline'     18
b'8155_6104_fiber_157_multiline'      18
b'8566_9102_fiber_958_multiline'      18
b'8338_3702_fiber_220_multiline'      18
b'8553_3701_fiber_254_multiline'      18
b'8550_12702_fiber_883_multiline'     18
b'8243_6101_fiber_451_multiline'      18
b'8132_9102_fiber_148_multiline'      17
b'8934_12704_fib

# False Positive Counts

In [51]:
inspection_points[should_be_0].id.value_counts(sort=True)

b'8550_12702_fiber_883_multiline'     74
b'8263_12704_fiber_212_multiline'     74
b'8318_9102_fiber_75_multiline'       73
b'8149_12702_fiber_669_multiline'     73
b'8606_6101_fiber_173_multiline'      72
b'8132_9102_fiber_958_multiline'      72
b'8465_12701_fiber_1009_multiline'    72
b'8947_6102_fiber_475_multiline'      72
b'8082_12704_fiber_1855_multiline'    71
b'8728_6104_fiber_683_multiline'      70
b'8254_12704_fiber_914_multiline'     70
b'8931_9101_fiber_580_multiline'      69
b'8934_12704_fiber_404_multiline'     69
b'8726_9101_fiber_758_multiline'      69
b'8258_6101_fiber_588_multiline'      69
b'8931_9101_fiber_124_multiline'      67
b'8131_3704_fiber_334_multiline'      66
b'8465_12701_fiber_120_multiline'     66
b'8155_6104_fiber_277_multiline'      65
b'8082_12704_fiber_202_multiline'     64
b'8553_3701_fiber_254_multiline'      63
b'8084_12701_fiber_1097_multiline'    63
b'8604_9101_fiber_614_multiline'      62
b'8083_9101_fiber_1334_multiline'     62
b'8132_9102_fibe

# False Negatives (Pred < .1)

In [10]:
false_negatives = inspection_points[less_than_10 & should_be_1].id.value_counts()
false_negatives

b'8252_6103_fiber_697_multiline'      28
b'8626_12704_fiber_554_multiline'     26
b'9026_9101_fiber_103_multiline'      25
b'8315_12702_fiber_932_multiline'     25
b'8606_6102_fiber_269_multiline'      24
b'8440_1902_fiber_141_multiline'      23
b'8615_1901_fiber_153_multiline'      22
b'8725_9101_fiber_1013_multiline'     21
b'8947_1901_fiber_142_multiline'      21
b'7975_12701_fiber_1694_multiline'    20
b'8313_6103_fiber_26_multiline'       20
b'8082_12702_fiber_296_multiline'     20
b'7443_12701_fiber_1791_multiline'    20
b'8485_6103_fiber_330_multiline'      20
b'8943_1901_fiber_97_multiline'       19
b'8987_6103_fiber_164_multiline'      19
b'8081_9101_fiber_628_multiline'      18
b'8604_12701_fiber_807_multiline'     18
b'8939_6102_fiber_635_multiline'      17
b'8332_12705_fiber_298_multiline'     17
b'8714_6103_fiber_25_multiline'       17
b'8726_1901_fiber_117_multiline'      17
b'9026_3703_fiber_242_multiline'      16
b'8252_6103_fiber_571_multiline'      16
b'7977_9101_fibe

In [11]:
# Printing out in a nice format.
for t in list(false_negatives.iteritems())[:20]:
    splitted = str(t[0]).split('_')
    splitted = splitted[:2] + splitted[3:4]
    print(' '.join(splitted).lstrip("'b") + ' ' + str(t[1]))

8252 6103 697 28
8626 12704 554 26
9026 9101 103 25
8315 12702 932 25
8606 6102 269 24
8440 1902 141 23
8615 1901 153 22
8725 9101 1013 21
8947 1901 142 21
7975 12701 1694 20
8313 6103 26 20
8082 12702 296 20
7443 12701 1791 20
8485 6103 330 20
8943 1901 97 19
8987 6103 164 19
8081 9101 628 18
8604 12701 807 18
8939 6102 635 17
8332 12705 298 17


## False Negative Counts

In [44]:
inspection_points[should_be_1].id.value_counts(sort=True)

b'8252_6103_fiber_697_multiline'      41
b'8626_12704_fiber_554_multiline'     34
b'9026_9101_fiber_103_multiline'      31
b'8252_6103_fiber_571_multiline'      31
b'8315_12702_fiber_932_multiline'     31
b'8947_1901_fiber_142_multiline'      29
b'7960_3701_fiber_162_multiline'      28
b'8606_6102_fiber_269_multiline'      28
b'7443_12701_fiber_1791_multiline'    28
b'8485_6103_fiber_330_multiline'      28
b'8313_6103_fiber_26_multiline'       27
b'8950_6103_fiber_186_multiline'      27
b'8604_12701_fiber_807_multiline'     27
b'8943_1901_fiber_97_multiline'       26
b'8440_1902_fiber_141_multiline'      26
b'8726_1901_fiber_117_multiline'      26
b'7977_9101_fiber_741_multiline'      25
b'8082_12702_fiber_296_multiline'     25
b'8485_6103_fiber_329_multiline'      24
b'8714_6103_fiber_25_multiline'       24
b'8439_12705_fiber_1059_multiline'    24
b'7990_1902_fiber_66_multiline'       23
b'8615_1901_fiber_153_multiline'      23
b'8140_3701_fiber_68_multiline'       23
b'8725_9101_fibe